In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

"""li.sports_list 내부의 a 태그 정보 추출"""
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = None
try:
    # 드라이버 설정
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=chrome_options
    )

    # 페이지 접속
    url = "https://wwwnew.kweather.co.kr/forecast/forecast_sports.html?idx=baseball"
    driver.get(url)
    logger.info("페이지 로드 완료")

    # 데이터 추출
    stadium_link_dic = {}
    stadium_list = []
    sports_list_items = driver.find_elements(By.CSS_SELECTOR, 'li.sports_list')
    logger.info(f"총 {len(sports_list_items)}개의 sports_list 항목 발견")

    for li_item in sports_list_items:
        try:
            # 각 li 내부의 span들에서 a 태그 찾기
            spans = li_item.find_elements(By.CSS_SELECTOR, 'span')
            for span in spans:
                try:
                    a_tag = span.find_element(By.CSS_SELECTOR, "a[href*='PouVSwRTRIN&idx=baseball']")
                    href = a_tag.get_attribute('href')
                    
                    try:
                        name = a_tag.text
                    except NoSuchElementException:
                        name = ""
                    stadium_list.append(name)
                    stadium_link_dic[name] = href

                    logger.debug(f"추출 완료: {name} - {href}")
                    
                except NoSuchElementException:
                    continue


        except Exception as e:
            logger.warning(f"항목 처리 중 오류: {str(e)}")
            continue

except Exception as e:
    logger.error(f"전체 프로세스 오류: {str(e)}")
finally:
    if driver:
        driver.quit()
        logger.info("드라이버 종료 완료")


print("\n[추출 결과]")
# stadium_list
stadium_link_dic

        

INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [C:\Users\user\.wdm\drivers\chromedriver\win64\135.0.7049.95\chromedriver-win32/chromedriver.exe] found in cache
INFO:__main__:페이지 로드 완료
INFO:__main__:총 1개의 sports_list 항목 발견
INFO:__main__:드라이버 종료 완료



[추출 결과]


{'대구야구장': 'https://wwwnew.kweather.co.kr/forecast/forecast_sports.html?index=1&area=27230510|143&rname=Xff3TxYgVPouVSwRTRIN&idx=baseball',
 '마산야구장': 'https://wwwnew.kweather.co.kr/forecast/forecast_sports.html?index=2&area=48160720|155&rname=XRL/UeMkVPouVSwRTRIN&idx=baseball',
 '사직야구장': 'https://wwwnew.kweather.co.kr/forecast/forecast_sports.html?index=3&area=26260590|159&rname=Xu0jUQj1VPouVSwRTRIN&idx=baseball',
 '목동야구장': 'https://wwwnew.kweather.co.kr/forecast/forecast_sports.html?index=4&area=11470510|108&rname=XRUgUPANVPouVSwRTRIN&idx=baseball',
 '잠실야구장': 'https://wwwnew.kweather.co.kr/forecast/forecast_sports.html?index=5&area=11710720|108&rname=XgkXUewYVPouVSwRTRIN&idx=baseball',
 '문학야구장': 'https://wwwnew.kweather.co.kr/forecast/forecast_sports.html?index=6&area=28170740|112&rname=XRcvUvYNVPouVSwRTRIN&idx=baseball',
 '월명야구장': 'https://wwwnew.kweather.co.kr/forecast/forecast_sports.html?index=7&area=45130680|140&rname=XgYLUAv5VPouVSwRTRIN&idx=baseball',
 '무등야구장': 'https://wwwnew.k

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd

"""
경기장 이름을 입력받아 해당 페이지의 날씨 정보 추출
:param stadium_link_dic: {'경기장명': '링크'} 형식의 딕셔너리
:param stadium_name: 조회할 경기장 이름
:return: 테이블 헤더와 데이터를 포함한 딕셔너리
"""
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = None

try:
    # 드라이버 설정
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=chrome_options
    )

    all_rows = []

    for stadium_name in stadium_list:

        # 해당 경기장 페이지 접속
        driver.get(stadium_link_dic[stadium_name])
        logger.info(f"{stadium_name} 페이지 로드 시작")

        # 테이블 로딩 대기 (최대 15초)
        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'li.lifestyle_present_forecast_table'))
            )
        except TimeoutException:
            logger.error("테이블 요소를 찾을 수 없음")
            

        # 테이블 데이터 추출
        table = driver.find_element(By.CSS_SELECTOR, 'li.lifestyle_present_forecast_table')
        
        # thead에서 컬럼명 추출
        columns = [th.text for th in table.find_elements(By.CSS_SELECTOR, 'thead th')]
        
        # tbody에서 데이터 추출
        rows = []
        for tr in table.find_elements(By.CSS_SELECTOR, 'tbody tr'):
            row_data = [td.text for td in tr.find_elements(By.TAG_NAME, 'td')]
            if row_data:  # 빈 행 제외
                rows.append(row_data)

        # 결과 구조화
        result = {
            'stadium': stadium_name,
            'columns': columns,
            'rows': rows
        }

        all_rows.extend(rows)

        logger.info(f"{stadium_name} 데이터 추출 완료")
        print(f"{stadium_name} 데이터 추출 완료")

except Exception as e:
    logger.error(f"전체 프로세스 오류: {str(e)}")

all_rows = [[stadium_list[i]] + row for i, row in enumerate(all_rows)]
all_columns = ['stadium'] + columns

df = pd.DataFrame(all_rows, columns=all_columns)

driver.quit()

df

INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [C:\Users\user\.wdm\drivers\chromedriver\win64\135.0.7049.95\chromedriver-win32/chromedriver.exe] found in cache
INFO:__main__:대구야구장 페이지 로드 시작
INFO:__main__:대구야구장 데이터 추출 완료


대구야구장 데이터 추출 완료


INFO:__main__:마산야구장 페이지 로드 시작
INFO:__main__:마산야구장 데이터 추출 완료


마산야구장 데이터 추출 완료


INFO:__main__:사직야구장 페이지 로드 시작
INFO:__main__:사직야구장 데이터 추출 완료


...

INFO:__main__:청주야구장 페이지 로드 시작
INFO:__main__:청주야구장 데이터 추출 완료


청주야구장 데이터 추출 완료


INFO:__main__:한밭야구장 페이지 로드 시작
INFO:__main__:한밭야구장 데이터 추출 완료


한밭야구장 데이터 추출 완료


,stadium,기온,체감,풍향,풍속,강수,습도
0,대구야구장,18.9℃,18.9℃,북,1.1m/s,-mm,63%
1,마산야구장,17.7℃,17.7℃,동남동,2.0m/s,-mm,81%
2,사직야구장,16.8℃,16.8℃,남서,7.7m/s,-mm,82%
3,목동야구장,18.7℃,18.7℃,남동,1.6m/s,-mm,71%
4,잠실야구장,18.7℃,18.7℃,남동,1.6m/s,-mm,71%
5,문학야구장,17.8℃,17.8℃,남,3.0m/s,-mm,76%
6,월명야구장,20.3℃,20.3℃,남동,3.5m/s,-mm,70%
7,무등야구장,19.5℃,19.5℃,남동,4.5m/s,-mm,71%
8,청주야구장,21.5℃,21.5℃,남남동,1.2m/s,-mm,54%
9,한밭야구장,21.4℃,21.4℃,남남동,1.8m/s,-mm,58%
